# Dynamic mputation RNN
### Prediction Imputation

- For this model we do not mask the training set's missing instances. We impute them.

- We do, however, perform masking for the teting data evaluation.

- Imputation with X_train, y_train
    - Divide into X_full and X_nan; y_full and y_nan
    - Use X_full and y_full to impute for X_nan and y_nan
- X_test, y_test
    - Impute with model learned from training data 

# Updates
- Remove all test operations
- mask_list.append((train, test)) went to mask_list.append(train)

### Functions

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn as sk
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold

def select_columns(col_list, n_months):
    """
    Takes in a list of column names and number of visits starting at 0.
    Returns column list time-stepped and dovetailed.
    """ 
    return dovetail_names(*[time_step_names(i, n_months) for i in col_list])
        
def time_step_names(name, n_months):

    return [(name + '_%d' % (j+1)) for j in range(-1,n_months*6, 6)]

def dovetail_names(*kwargs):
    """
    Dovetails column names across time slices cccording to preset order.
    """
    zipped = zip(*kwargs)
    l = []
   
    for i in zipped:
        
        for j in i:
            
            l.append(j)
            
    return l

def reshape_data(X, y, n_time_steps, n_features = 115):  
    
    X_reshaped = X.values.reshape(-1, n_time_steps, n_features)
    y_reshaped = y.values.reshape(-1, n_time_steps, 1)
    print("X reshaped is " + str(X_reshaped.shape))
    print("y reshaped is " + str(y_reshaped.shape))

    y = y_reshaped.astype(float)
    X = X_reshaped.astype(float)
    
    print(X.shape, y.shape)
    return X, y

def provide_data(X,y,roll,n_features=115):
 
    X = X.iloc[:,:n_features*roll]
    y = y.iloc[:,:roll]

    y_full = y.dropna()

    mask = X.index.isin(y_full.index.tolist())

    X_full = X[mask]

    y_nan = y[~mask]
    X_nan = X[~mask]
    
    print('NaN')
    X_nan, y_nan = reshape_data(X_nan, y_nan, roll)
    print('Full')
    X_full, y_full = reshape_data(X_full, y_full, roll)
    
    return X_full, X_nan, y_full, y_nan, mask


def provide_all_data(X,y,roll):
 
    X = X.iloc[:,:n_features*roll]
    y = y.iloc[:,:roll]

    X_all, y_all = reshape_data(X, y, roll)
    
    return X_all, y_all
    
def prepare_for_mask(X,y):
    
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            if np.isnan(y[i][j][0]) or (y[i][j] == -99):

                X[i][j] = -99
                y[i][j] = -99
                
    return X,y

/Users/matiasberretta/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Keras Callbacks

In [2]:
weight_file_path = "weights/my_model_weights_6m|6m.h5"
final_file_path = "final_weights/my_model_weights_6m|6m.h5"
best_file_path = "best/best_weights_6m|6m.hdf5"

In [3]:
from keras import callbacks
import keras.layers as L
import keras.models as M
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error as mse

# reduce learning rate on plateau
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                               patience=5, min_lr=0.001)

# stop training if there isn't a significant improvement in the course of 5 epochs
early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, 
                              patience=5, verbose=0, mode='auto', 
                              baseline=None, restore_best_weights=True)

# model check point
model_checkpoint = ModelCheckpoint(best_file_path, monitor='val_loss', 
                                   verbose=1, 
                                   save_best_only=True, 
                                   mode='min')

callbacks_list = [reduce_lr, early_stopping]
callbacks_list_final = [reduce_lr, early_stopping, model_checkpoint]

Using TensorFlow backend.


In [4]:
def generate_initial_predictions(n_time_steps, X_train, X_test, y_train, y_test, n_inputs=115):
    
    threshold = len(y_train.columns)
    print("TRAIN")
    X_full_train, X_nan_train, y_full_train, y_nan_train, mask_train = provide_data(X_train, y_train, n_time_steps)
    print("TEST")
    X_full_test, X_nan_test, y_full_test, y_nan_test, mask_test = provide_data(X_test, y_test, n_time_steps)
    
    # This is irrelevant because "full" data sets contain no missing values to mask
    masked_X_full_test, masked_y_full_test = prepare_for_mask(X_full_test, y_full_test)
    masked_X_full_train, masked_y_full_train = prepare_for_mask(X_full_train, y_full_train)
    
    K.clear_session()
    
    m = M.Sequential()
    m.add(L.Masking(mask_value=-99, input_shape=(n_time_steps, n_inputs)))
    m.add(L.GRU(128, return_sequences=True))
    m.add(L.Dense(1, activation='relu'))
    m.compile(optimizer = 'nadam', loss = 'mean_absolute_error')

    if n_time_steps > 1:
        
        # load weights from previous model to establish continuity 
        m.load_weights(weight_file_path)
    
    history = m.fit(masked_X_full_train, masked_y_full_train, 
                    validation_split=0.2,
                    batch_size = masked_X_full_train.shape[0], # batch size == size of the training data 
                    epochs=1, 
                    shuffle=True,
                    callbacks = callbacks_list)
        
    m.save_weights(weight_file_path) # save weights 
    
    # Predict for unknown values of training and testing data
    # Using train data with known targets, I train a model to predict for missing values
    # In both training and testing data 
    # But for the testing data, I only impute the X feature space 
    
    m_pred_train = pd.Series(np.array([x[0] for x in m.predict(X_nan_train)]).flatten(),
                             index = y_train[~mask_train].index)
    
    # redundant but also helps me known which values to update for test data feature space
    m_pred_test = pd.Series(np.array([x[0] for x in m.predict(X_nan_test)]).flatten(),
                           index = y_test[~mask_test].index)
    
    # In pandas the best way to update a dataframe or series
    # is with another dataframe or series with the desired index
    
    m_pred_train_y = m_pred_train.copy()
    m_pred_train_y[:] = -99
    
    y_train.iloc[:,n_time_steps-1].update(m_pred_train_y)
    
    if (n_time_steps < threshold): # when y contains an EDSS column that X does not 
        
        X_train.iloc[:,(n_inputs*(n_time_steps+1))-1].update(m_pred_train)
        # redundant but also helps me known which values to update for test data feature space
        X_test.iloc[:,(n_inputs*(n_time_steps+1))-1].update(m_pred_test)
        
    if (n_time_steps == threshold):
        m.save_weights(final_file_path)
        
    X_train_all, y_train_all = provide_all_data(X_train, y_test, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    masked_X_train_all, masked_y_train_all = prepare_for_mask(X_train_all, y_train_all)
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    # predict for all values 
    # use the masking list to known which of these should be used as imputation values
    all_train_pred = m.predict(masked_X_train_all)
    all_test_pred = m.predict(masked_X_test_all)

    return X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test 

In [5]:
def update_previous_predictions(n_time_steps, all_train_pred, all_test_pred):
    
    m_pred_train = np.array(all_train_pred[:,n_time_steps-1,:]).flatten()
    m_pred_test = np.array(all_test_pred[:,n_time_steps-1,:]).flatten()
    
    return m_pred_train, m_pred_test

In [6]:
# should have a way of identifying which isntances to update
def impute_previous_predictions(n_time_steps, X_train, X_test, 
                                m_pred_train, m_pred_test, mask_list, n_inputs=115):
    
    mask_train = mask_list[n_time_steps-1][0]
    mask_test = mask_list[n_time_steps-1][1]
    
    m_pred_train = pd.Series(m_pred_train[~mask_train], index = y_train[~mask_train].index)
    m_pred_test = pd.Series(m_pred_test[~mask_test], index = y_test[~mask_test].index)
    
    if n_time_steps < len(y_train.columns): # 36
        
        X_train.iloc[:,(n_inputs*(n_time_steps+1))-1].update(m_pred_train)
        X_test.iloc[:,(n_inputs*(n_time_steps+1))-1].update(m_pred_test)
    
    return X_train, X_test

# Import Data

In [7]:
X = pd.read_csv('../../../data/pre_imputation_data/X_6_months|6_months.csv', index_col = 0)
y = pd.read_csv('../../../data/pre_imputation_data/y_6_months|6_months.csv', index_col = 0)

#n_time_steps = len(y.columns)
n_features = X.columns.tolist().index("EDSS_0")+1
pot = 1
n_inputs = n_features * pot
n_units = 128
skf = KFold(n_splits=5, shuffle=True, random_state=42)
print("The input length of the training data will be", pot, "time slices, separated by 6 month intervals")
print(n_features, "featues comprise one time slice")

The input length of the training data will be 1 time slices, separated by 6 month intervals
115 featues comprise one time slice


In [8]:
# Drop all missing values from EDSS_0 which is our first X feature predictor
mask = X.index.isin(X['EDSS_0'].dropna().index.tolist())

X_og, y_og = X[mask], y[mask]

In [9]:
train_ix = []
test_ix = []
for train, test in skf.split(X_og.index):
    train_ix.append(train)
    test_ix.append(test)

In [10]:
def def_train_name(f_ix):
    
    X_train_name = "X_train_f" + str(f_ix + 1) + ".csv"
    y_train_name = "y_train_f" + str(f_ix + 1) + ".csv"
    X_test_name = "X_test_f" + str(f_ix + 1) + ".csv"
    y_test_name = "y_test_f" + str(f_ix + 1) + ".csv"
    
    return X_train_name, X_test_name, y_train_name, y_test_name

In [11]:
f_ix = 4

X_train_og, y_train_og = X_og.iloc[train_ix[f_ix], :].copy(), y_og.iloc[train_ix[f_ix], :].copy()
X_test_og, y_test_og = X_og.iloc[test_ix[f_ix], :].copy(), y_og.iloc[test_ix[f_ix], :].copy()

# Divide Data into Training and Testing

In [12]:
mask_list = []

X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = generate_initial_predictions(1, X_train_og, X_test_og,  y_train_og, y_test_og)

mask_list.append((mask_train, mask_test)) # these tell me what values are missing in each column 

TRAIN
NaN
X reshaped is (60, 1, 115)
y reshaped is (60, 1, 1)
(60, 1, 115) (60, 1, 1)
Full
X reshaped is (333, 1, 115)
y reshaped is (333, 1, 1)
(333, 1, 115) (333, 1, 1)
TEST
NaN
X reshaped is (17, 1, 115)
y reshaped is (17, 1, 1)
(17, 1, 115) (17, 1, 1)
Full
X reshaped is (81, 1, 115)
y reshaped is (81, 1, 1)
(81, 1, 115) (81, 1, 1)
Train on 266 samples, validate on 67 samples
Epoch 1/1
266/266 [==============================] - 2s 8ms/step - loss: 1.7572 - val_loss: 1.0009
X reshaped is (393, 1, 115)
y reshaped is (98, 1, 1)
(393, 1, 115) (98, 1, 1)
X reshaped is (98, 1, 115)
y reshaped is (98, 1, 1)
(98, 1, 115) (98, 1, 1)


In [13]:
print("There are missing values for EDSS in the X feature space:", pd.isna(X_train['EDSS_6']).any())
print("There are missing values for EDSS in the y target space:", pd.isna(y_train['EDSS_6']).any())

There are missing values for EDSS in the X feature space: False
There are missing values for EDSS in the y target space: False


In [14]:
for r in range(2, 37):
    
    X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = generate_initial_predictions(r, X_train, X_test,y_train, y_test)
    
    mask_list.append((mask_train, mask_test))

    if (all_train_pred.shape[1] > 1):
        
        # update all previous predictions
        for col in range(1,all_train_pred.shape[1]):
            # for each column of the training and testing EDSS
            m_pred_train, m_pred_test = update_previous_predictions(col, all_train_pred, all_test_pred)    
            X_train, X_test = impute_previous_predictions(col, X_train, X_test, m_pred_train, m_pred_test, mask_list)


TRAIN
NaN
X reshaped is (33, 2, 115)
y reshaped is (33, 2, 1)
(33, 2, 115) (33, 2, 1)
Full
X reshaped is (360, 2, 115)
y reshaped is (360, 2, 1)
(360, 2, 115) (360, 2, 1)
TEST
NaN
X reshaped is (22, 2, 115)
y reshaped is (22, 2, 1)
(22, 2, 115) (22, 2, 1)
Full
X reshaped is (76, 2, 115)
y reshaped is (76, 2, 1)
(76, 2, 115) (76, 2, 1)
Train on 288 samples, validate on 72 samples
Epoch 1/1
288/288 [==============================] - 2s 6ms/step - loss: 1.3893 - val_loss: 1.0831
X reshaped is (393, 2, 115)
y reshaped is (98, 2, 1)
(393, 2, 115) (98, 2, 1)
X reshaped is (98, 2, 115)
y reshaped is (98, 2, 1)
(98, 2, 115) (98, 2, 1)
TRAIN
NaN
X reshaped is (56, 3, 115)
y reshaped is (56, 3, 1)
(56, 3, 115) (56, 3, 1)
Full
X reshaped is (337, 3, 115)
y reshaped is (337, 3, 1)
(337, 3, 115) (337, 3, 1)
TEST
NaN
X reshaped is (33, 3, 115)
y reshaped is (33, 3, 1)
(33, 3, 115) (33, 3, 1)
Full
X reshaped is (65, 3, 115)
y reshaped is (65, 3, 1)
(65, 3, 115) (65, 3, 1)
Train on 269 samples, valida

X reshaped is (393, 14, 115)
y reshaped is (98, 14, 1)
(393, 14, 115) (98, 14, 1)
X reshaped is (98, 14, 115)
y reshaped is (98, 14, 1)
(98, 14, 115) (98, 14, 1)
TRAIN
NaN
X reshaped is (147, 15, 115)
y reshaped is (147, 15, 1)
(147, 15, 115) (147, 15, 1)
Full
X reshaped is (246, 15, 115)
y reshaped is (246, 15, 1)
(246, 15, 115) (246, 15, 1)
TEST
NaN
X reshaped is (82, 15, 115)
y reshaped is (82, 15, 1)
(82, 15, 115) (82, 15, 1)
Full
X reshaped is (16, 15, 115)
y reshaped is (16, 15, 1)
(16, 15, 115) (16, 15, 1)
Train on 196 samples, validate on 50 samples
Epoch 1/1
196/196 [==============================] - 2s 10ms/step - loss: 0.9044 - val_loss: 0.9194
X reshaped is (393, 15, 115)
y reshaped is (98, 15, 1)
(393, 15, 115) (98, 15, 1)
X reshaped is (98, 15, 115)
y reshaped is (98, 15, 1)
(98, 15, 115) (98, 15, 1)
TRAIN
NaN
X reshaped is (137, 16, 115)
y reshaped is (137, 16, 1)
(137, 16, 115) (137, 16, 1)
Full
X reshaped is (256, 16, 115)
y reshaped is (256, 16, 1)
(256, 16, 115) (256

Train on 75 samples, validate on 19 samples
Epoch 1/1
75/75 [==============================] - 2s 22ms/step - loss: 1.2164 - val_loss: 0.6916
X reshaped is (393, 27, 115)
y reshaped is (98, 27, 1)
(393, 27, 115) (98, 27, 1)
X reshaped is (98, 27, 115)
y reshaped is (98, 27, 1)
(98, 27, 115) (98, 27, 1)
TRAIN
NaN
X reshaped is (324, 28, 115)
y reshaped is (324, 28, 1)
(324, 28, 115) (324, 28, 1)
Full
X reshaped is (69, 28, 115)
y reshaped is (69, 28, 1)
(69, 28, 115) (69, 28, 1)
TEST
NaN
X reshaped is (95, 28, 115)
y reshaped is (95, 28, 1)
(95, 28, 115) (95, 28, 1)
Full
X reshaped is (3, 28, 115)
y reshaped is (3, 28, 1)
(3, 28, 115) (3, 28, 1)
Train on 55 samples, validate on 14 samples
Epoch 1/1
55/55 [==============================] - 2s 34ms/step - loss: 0.6324 - val_loss: 1.1221
X reshaped is (393, 28, 115)
y reshaped is (98, 28, 1)
(393, 28, 115) (98, 28, 1)
X reshaped is (98, 28, 115)
y reshaped is (98, 28, 1)
(98, 28, 115) (98, 28, 1)
TRAIN
NaN
X reshaped is (334, 29, 115)
y re

# X_train should go up to 210

In [15]:
save_temp = (X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test)
save_list_temp = mask_list.copy()

In [16]:
X_train, X_test, y_train, y_test, all_train_pred, all_test_pred, mask_train, mask_test = save_temp 
mask_list = save_list_temp

# Try 

In [17]:
def final_model(X_train, y_train, X_test, y_test, n_inputs=115):
    
    """Running the model after we've impute for the whole feature space"""
    
    n_time_steps = len(y_train.columns)
    
    X_train_all, y_train_all = provide_all_data(X_train, y_train, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    # we do want to mask the instances of the testing data where there are missing targets
    # so that it does not affect our val_loss
    masked_X_train_all, masked_y_train_all = prepare_for_mask(X_train_all, y_train_all)
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    K.clear_session()

    m = M.Sequential()
    m.add(L.Masking(mask_value=-99, input_shape=(n_time_steps, n_inputs)))
    m.add(L.GRU(128, return_sequences=True))
    m.add(L.Dense(1, activation='relu'))
    m.compile(optimizer = 'nadam', loss = 'mean_absolute_error')

    # load weights from previous model to establish continuity 
    m.load_weights(final_file_path)
    
    history = m.fit(masked_X_train_all, masked_y_train_all,
                    validation_split = 0.2, 
                    batch_size = X_train_all.shape[0], # of training instances == batch_size 
                    epochs=1,
                    shuffle=True,
                   callbacks = callbacks_list_final) # should save best weights 
    
    m.save_weights(final_file_path)
    
    all_train_pred = m.predict(X_train_all)
    all_test_pred = m.predict(masked_X_test_all)
    
    val_loss = history.__dict__['history']['val_loss']
    
    return all_train_pred, all_test_pred, val_loss, m

In [18]:
def run_model(r, X_train, y_train, X_test, y_test):
    """We need to divide our RNN into different 1 epoch models in order to iteratively update the imputations"""
    val_loss_list = []
    
    for i in range(r):
        
        all_train_pred, all_test_pred, val_loss, model = final_model(X_train, y_train, X_test, y_test)

        val_loss_list.append(val_loss)
    
        for i in range(1, all_train_pred.shape[1]): # -1 

                    m_pred_train, m_pred_test = update_previous_predictions(i, all_train_pred, all_test_pred)    
                    
                    X_train, X_test = impute_previous_predictions(i, X_train, X_test, m_pred_train, m_pred_test, mask_list)
    
    return np.min(val_loss_list), model

# Replace test and train y with original copies to mask missing target values

In [19]:
best, model = run_model(500, X_train, y_train_og, X_test, y_test_og) # use original y test

X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 3s 8ms/step - loss: 1.0313 - val_loss: 0.7479

Epoch 00001: val_loss improved from inf to 0.74793, saving model to best/best_weights_6m|6m.hdf5
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.7731 - val_loss: 1.0410

Epoch 00001: val_loss did not improve from 0.74793
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [======

Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.6050 - val_loss: 1.2106

Epoch 00001: val_loss did not improve from 0.70251
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 1.2280 - val_loss: 0.7281

Epoch 00001: val_loss did not improve from 0.70251
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.6034 - val_loss: 0.8279

Epoch 00001: val_loss did not improve from 0.70251
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)


X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.7181 - val_loss: 0.9389

Epoch 00001: val_loss did not improve from 0.67967
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.8842 - val_loss: 0.7376

Epoch 00001: val_loss did not improve from 0.67967
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss


Epoch 00001: val_loss did not improve from 0.67967
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.8830 - val_loss: 0.7129

Epoch 00001: val_loss did not improve from 0.67967
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.6766 - val_loss: 0.9623

Epoch 00001: val_loss did not improve from 0.67967
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [

Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.5863 - val_loss: 1.0598

Epoch 00001: val_loss did not improve from 0.66235
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 1.0016 - val_loss: 0.6848

Epoch 00001: val_loss did not improve from 0.66235
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.5316 - val_loss: 0.9648

Epoch 00001: val_loss did not improve from 0.66235
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)


...

Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.6312 - val_loss: 0.7699

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4581 - val_loss: 0.7755

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.6226 - val_loss: 0.7928

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)


Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.5526 - val_loss: 0.7166

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4072 - val_loss: 0.8550

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.5398 - val_loss: 0.7371

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)


Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.5852 - val_loss: 0.7273

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4782 - val_loss: 0.8931

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.5689 - val_loss: 0.7252

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)


Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4771 - val_loss: 0.8522

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.5669 - val_loss: 0.7330

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4716 - val_loss: 0.8592

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)


Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4774 - val_loss: 0.7773

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.3689 - val_loss: 0.7419

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)
Train on 314 samples, validate on 79 samples
Epoch 1/1
314/314 [==============================] - 2s 6ms/step - loss: 0.4749 - val_loss: 0.8323

Epoch 00001: val_loss did not improve from 0.65983
X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)


In [20]:
best

0.6598294973373413

In [21]:
def best_model(X_train, y_train, X_test, y_test, n_inputs=115):
    
    """Running the model after we've impute for the whole feature space"""
    
    n_time_steps = len(y_train.columns)
    
    X_train_all, y_train_all = provide_all_data(X_train, y_train, n_time_steps)
    X_test_all, y_test_all = provide_all_data(X_test, y_test, n_time_steps)
    
    # we do want to mask the instances of the testing data where there are missing targets
    # so that it does not affect our val_loss
    masked_X_test_all, masked_y_test_all = prepare_for_mask(X_test_all, y_test_all)
    
    K.clear_session()
    m = M.Sequential()
    m.add(L.Masking(mask_value=-99, input_shape=(n_time_steps, n_inputs)))
    m.add(L.GRU(128, return_sequences=True))
    m.add(L.Dense(1, activation='relu'))
    m.compile(optimizer = 'nadam', loss = 'mean_absolute_error')

    # load weights from previous model to establish continuity 
    m.load_weights(best_file_path)
    
    all_train_pred = m.predict(X_train_all)
    all_test_pred = m.predict(masked_X_test_all)
    
    return all_train_pred, all_test_pred, m

In [22]:
def return_imputed_data(X_train, y_train, X_test, y_test):
    """Imputes for test data Feature EDSS with best model weights and then evaluates test data"""
    val_loss_list = []
    
    all_train_pred, all_test_pred, m = best_model(X_train, y_train, X_test, y_test)
    
    for i in range(1, all_train_pred.shape[1]): # -1 

                m_pred_train, m_pred_test = update_previous_predictions(i, all_train_pred, all_test_pred)    
                    
                X_train, X_test = impute_previous_predictions(i, X_train, X_test, m_pred_train, m_pred_test, mask_list)
    
    return X_train, X_test, y_train, y_test

In [23]:
X_train, X_test, y_train, y_test = return_imputed_data(X_train, y_train_og, X_test, y_test_og)

X reshaped is (393, 36, 115)
y reshaped is (393, 36, 1)
(393, 36, 115) (393, 36, 1)
X reshaped is (98, 36, 115)
y reshaped is (98, 36, 1)
(98, 36, 115) (98, 36, 1)


In [24]:
a,b,c,d = def_train_name(f_ix)

X_train.to_csv(a)
X_test.to_csv(b)
y_train.to_csv(c)
y_test.to_csv(d)

## FIRST PART
- FIRST IMPUTATION ONE TIME STEP AT A TIME
- WE USE ONLY THE ORIGINAL TRAINING DATA and DO A VALIDATION SPLIT TO TUNE PARAMETERS

## SECOND PART
- WE START THIS PART ONCE WE HAVE IMPUTED OUR WHOLE FEATURE SPACE
- WE RUN OUR MODEL FOR 100 EPOCHS
- WE EVALUATE ON THE DATA WE LEFT ASIDE


ORIGINAL TRAIN -> TRAINING and VALIDATION TO TUNE PARAMETERS

ORIGINAL TEST -> FINAL VALUE 

The only thing that gets updated is the training operation with known training data
but we impute for the testing data at each epoch once we have imputed the whole feature space 